In [2]:
import psycopg2
import pandas as pd
import json

C:\Users\valen\AppData\Local\Temp\ipykernel_1948\1122502516.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [8]:
def connect_to_db():
    db_conn = None
    try:
        with open('C:/Users/valen/OneDrive/Documentos/workshop_1/Workshop_1/loading_to_postgres/db_config.json', 'r') as config_file:
            db_settings = json.load(config_file)

        db_conn = psycopg2.connect(
            host='localhost',
            user=db_settings['user'],
            password=db_settings['password'],
            dbname=db_settings['database']
        )
        print('Connection to the database was successful')
    except psycopg2.DatabaseError as db_error:
        print('Failed to connect to the database:', db_error)
    return db_conn

connect_to_db()

Connection to the database was successful


<connection object at 0x0000023EA5165250; dsn: 'user=postgres password=xxx dbname=workshop3 host=localhost', closed: 0>

In [9]:
def get_hires_by_technology(db_conn):
    # Creamos un objeto cursor utilizando la conexión a la base de datos
    cursor = db_conn.cursor()
    
    # Definimos la consulta SQL para contar los contratados agrupados por tecnología
    query = """
    SELECT tech_stack, COUNT(*) 
    FROM applicants 
    WHERE hired_status = True 
    GROUP BY tech_stack;
    """
    
    try:
        cursor.execute(query)
        hires_by_tech = cursor.fetchall()
        df_hires_by_tech = pd.DataFrame(hires_by_tech, columns=['Technology', 'Hires'])
        return df_hires_by_tech
    except psycopg2.DatabaseError as e:
        print("Database error:", e)
    finally:
        cursor.close()

db_conn = connect_to_db()
if db_conn:
    hires_by_technology_df = get_hires_by_technology(db_conn)
    db_conn.close()  # Cerramos la conexión cuando terminamos
    print(hires_by_technology_df)  # Imprimimos el DataFrame obtenido
else:
    print("No se pudo establecer una conexión con la base de datos.")

Connection to the database was successful
                                 Technology  Hires
0                 Development - CMS Backend    284
1                             Data Engineer    255
2                                Salesforce    256
3                                     Sales    239
4                             QA Automation    243
5                                    DevOps    495
6                                 QA Manual    259
7                     System Administration    293
8                   Development - FullStack    254
9                   Database Administration    282
10                    Business Intelligence    254
11               Development - CMS Frontend    251
12                         Game Development    519
13                      Security Compliance    250
14                           Client Success    271
15                    Development - Backend    255
16                   Development - Frontend    266
17                                 Secur